# 1-NN using 3 metrics: Manhattan, Euclidian and Chessboard

First of all let's import the dataset

In [1]:
#The dataset will be saved in variable: df
import csv
#path of my file
p= "D:\\CIC\\CIP\\Tareas\\Tarea_12\\sonar.csv"

with open(p, newline='') as csvfile:
    
    # create a CSV reader object
    csvreader = csv.reader(csvfile, delimiter=',')
    
    # create a list to hold the rows of the CSV file
    df = []
    
    # loop through each row in the CSV file
    for row in csvreader:
        
        # convert the row to a list of numbers
        num_row = [float(x) for x in row]
        
        # add the num_row to the list of rows
        df.append(num_row)
        
# print the 2D list
print(len(df))

208


In [4]:
def balanced_df(df,class_1,class_2):
    n= len(df[0])-1 #suppose the class column is the last element in the vector
    
    counter_1 = 0
    counter_2 = 0
    
    for x in df:
        if x[n] == class_1:
            counter_1 += 1
        else:
            counter_2 += 1
    
    if max(counter_1,counter_2)/min(counter_1,counter_2)>1.5:
        print(f'Desbalanced Data Set: {max(counter_1,counter_2)}/{min(counter_1,counter_2)}={max(counter_1,counter_2)/min(counter_1,counter_2)}')
        if max(counter_1,counter_2)==counter_1:
            print(f"The bigger class is the: {class_1}")
        else:
            print(f"The bigger class is the: {class_2}")
    else:
        print(f'Balanced Data Set: {max(counter_1,counter_2)}/{min(counter_1,counter_2)}={max(counter_1,counter_2)/min(counter_1,counter_2)}')
        if max(counter_1,counter_2)==counter_1:
            print(f"The bigger class is the: {class_1}")
        else:
            print(f"The bigger class is the: {class_2}")

In [5]:
balanced_df(df,1,2)

Balanced Data Set: 111/97=1.1443298969072164
The bigger class is the: 1


The next code will help us to check if there're repeated or redundant patterns

In [2]:
import pandas as pd

#this function helps to clean the dataset

def process_dataset(lst):
    df=pd.DataFrame(lst)
    # Remove columns where all entries are the same
    df = df.loc[:, (df != df.iloc[0]).any()]

    # Remove duplicate rows
    df.drop_duplicates(inplace=True)

    # Remove rows with duplicate values in all but the last column
    df_without_last = df.iloc[:, :-1]
    df.drop_duplicates(subset=df_without_last.columns.tolist(), inplace=True, keep=False)

    arr = df.values.tolist()

    return arr

In [3]:
df=process_dataset(df)
print(len(df))

208


### Note that the dataset was already clean

### Let's define a function of Minkowski metric

In [3]:
# a and b should be vectors of the same lenght to compute the distance

# k is the value I want for the metric of Minkowski

def mink_metric(k,a,b):
    
    n=len(a)
    
    s=0 #sum
    
    for i in range(n):
        value= (abs(a[i]-b[i]))**k
        
        s=s+value
        
    s= s**(1/k)
    
    return s

In [4]:
a=[2,3]
b=[5,7]

s=mink_metric(5,a,b)
print(s)

4.174027662897746


### Let's define a function that compute the Chebyshev/Chessboard distance

In [5]:
# a and b should be vectors of the same lenght to compute the distance

# k is the value I want for the metric of Minkowski

def chebyshev_metric(a,b):
    
    n=len(a)
    
    values=[]
    
    for i in range(n):
        value= (abs(a[i]-b[i]))
        
        values.append(value)
        
    s= max(values)
    
    return s

In [6]:
a=[2,3]
b=[5,7]

s=chebyshev_metric(a,b)

print(s)

4


### Let's procced to do the 1-NN with LOOCV

### GOAL: 1-NN with LOOCV from the data set, compute the confusion matrix using 3 different metrics

### METRIC: City Block/Manhattan

In [7]:
num_attributes= len(df[0])

last_pos_index = 0

for i in range(len(df)):
    if df[i][num_attributes-1]==1:
        last_pos_index=i

print(last_pos_index)

110


In [8]:
#remove the last column of the dataset
for row in df:
    row.pop()

In [13]:
from collections import Counter
import math

#K-NN with Leave One Out Cross Validation
def knn_LOOCV(k,minkowski_value,df_validation,class_positive,class_negative,index_test,last_positive_index,TP,FN,FP,TN):
    
    train_df = df_validation #dataset where the first patterns are all positive class and the second part are all negative class

    pattern_index = index_test
    test_df = df_validation[pattern_index]

    n=last_positive_index
    
    if pattern_index<=n:
        class_of_test_pattern = class_positive
    else:
        class_of_test_pattern = class_negative

    distances = []

    for i in range(len(train_df)):
        
        #the lines below correspond to different distances 
        
        #dist = round(math.dist(test_df, train_df[i]),4) #Euclidian metric
        #dist = round(mink_metric(minkowski_value,test_df,train_df[i]),4) #minkowski metric
        dist = round(chebyshev_metric(test_df,train_df[i]),4) #Chebyshev metric

        if i<=n:
            distances.append([dist,i+1,class_positive]) #it need to be sum one because of the python numeration
        else:
            distances.append([dist,i+1,class_negative])

    del distances[pattern_index] #delete the list of distance from the test pattern to itself       

    kNN=k
    sorted_dist = sorted(distances, key=lambda x: x[0])[:kNN]



    #now i want to count the most repeated class from the KNN list

    # extract the third element from each sublist and put them in a separate list
    third_elements = [sublist[2] for sublist in sorted_dist]

    # use Counter to count the frequency of each value in the third_elements list
    counter = Counter(third_elements)

    most_common_class = counter.most_common(1)[0][0] #this print the most common class

    
    
    if most_common_class == class_of_test_pattern:
        
        
        if class_of_test_pattern==class_positive:
            TP +=1
        elif class_of_test_pattern==class_negative:
            TN +=1
    else:
        #print("\nFue un \\textbf{ERROR}")

        if class_of_test_pattern==class_positive:
            FN +=1
        elif class_of_test_pattern==class_negative:
            FP +=1
    return TP,FN,FP,TN

### We're going to suppose that the positive class is 1 and negative is 2

In [11]:
class_positive=1
class_negative=2

n=last_pos_index

TP=0
FN=0
FP=0
TN=0

#minkowski with m=1
for i in range(len(df)):
    TP,FN,FP,TN=knn_LOOCV(1,1,df,class_positive,class_negative,i,n,TP,FN,FP,TN)

    
CM=[[TP,FN],[FP,TN]]
print(CM)

[[98, 13], [18, 79]]


### METRIC: Euclidian

In [12]:
class_positive=1
class_negative=2

n=last_pos_index

TP=0
FN=0
FP=0
TN=0

#euclidian metric
for i in range(len(df)):
    TP,FN,FP,TN=knn_LOOCV(1,2,df,class_positive,class_negative,i,n,TP,FN,FP,TN)

CM=[[TP,FN],[FP,TN]]
print(CM)

[[96, 15], [21, 76]]


### Metric: Chebyshev

In [14]:
class_positive=1
class_negative=2

n=last_pos_index

TP=0
FN=0
FP=0
TN=0

for i in range(len(df)):
    TP,FN,FP,TN=knn_LOOCV(1,1,df,class_positive,class_negative,i,n,TP,FN,FP,TN)

CM=[[TP,FN],[FP,TN]]
print(CM)

[[96, 15], [28, 69]]


In [19]:
def pres(CM):
    TP = CM[0][0]
    FN = CM[0][1]
    
    FP = CM[1][0]
    TN = CM[1][1] 
    
    preci = TP/(TP+FP)
    
    return preci

def f1(CM):
    TP = CM[0][0]
    FN = CM[0][1]
    
    FP = CM[1][0]
    TN = CM[1][1] 
    
    f1 = (2*TP)/(2*TP+FN+FP)
    
    return f1
    
def mcc(CM):
    TP = CM[0][0]
    FN = CM[0][1]
    
    FP = CM[1][0]
    TN = CM[1][1] 
    
    mcc = (TP*FP-FP*FN)/((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    
    return mcc
    

def medidas(CM):
    accuracy = (CM[0][0]+CM[1][1])/(sum(CM[0])+sum(CM[1]))
    recall = (CM[0][0]) / (sum(CM[0]))
    specificity = (CM[1][1])/sum(CM[1])
    balanced_acc= (recall+specificity)/2
    precision = pres(CM)
    f1_ = f1(CM)
    mcc_ = mcc(CM) 
    
    print(f"\\item Accuracy:{round(accuracy,4)}\n \\item Recall:{round (recall,4)}\n \\item Specificity: {round(specificity,4)}\n \\item Balanced Accuracy:{round(balanced_acc,4)}\n \\item Precision: {round(precision,4)}\n \\item f1: {round(f1_,4)}\n \\item MCC: {round(mcc_,4)}")

In [22]:
CM=[[96, 15], [28, 69]]
medidas(CM)

\item Accuracy:0.7933
 \item Recall:0.8649
 \item Specificity: 0.7113
 \item Balanced Accuracy:0.7881
 \item Precision: 0.7742
 \item f1: 0.817
 \item MCC: 0.0
